# Scope mimicry, Python version

to diagnose potential trigger and buffer storage issue

Pierre, March 2025

In [23]:
import numpy as np
from enum import Enum
F = False
T = True

## ScopeMimicry code

### Original C++ version adapted to Python

In [2]:
class ScopeMimicry:
    def __init__(self, length: int, nb_channel: int):
        """
    	uint16_t _idx_name;
    	uint16_t _idx_datas;
    	float32_t *_channel_numbers;
    	float32_t *_memory;
    
    	uint16_t _decimation;

    	/* variable for dump_datas method */
    	e_dump_state dump_state;
    	char hash[2] = "#";
    	char nullchar[2] = " ";
    	char char_name[256];
    	/* each data is a float written in hexa so eight chars with \n and \0 chars at end */
    	char char_data[10];
    	char *data_dumped;
        """
        print(f"Init ScopeMimicry(length={length}, nb_channel={nb_channel})")
        self._length = length
        self._nb_channel = nb_channel
        self._acq_counter = 0
        self._effective_chan_number = 0
        self._old_trigg_value = False
        self._trigged = False
        self._delay = 0
        self._delay_complement = length
        self._trigged_counter = length
        self._final_idx = length-1

        self._memory = np.zeros(self._length*self._nb_channel)
        self._names = [f"ch{i}" for i in range(nb_channel)] # default name
        # Pointers to the global memory. 
        # Python-specific adaptation: values should be keys of the `global_memory` dict
        self._channels = [None for i in range(nb_channel)] 

        self._triggFunc = lambda : False
        # Python specific: mockup program global memory where the _channels pointer should point to
        self.global_memory = dict()
    # end init
    
    def connectChannel(self, channel_addr, name: str):
        """store channel address withing (Python-specific: in C it stores the pointer)
        along with channel name
        """
        print(f'Connect channel named "{name}" to variable at addr "{channel_addr}"')
        if self._effective_chan_number < self._nb_channel:
            # capture the reference of the variable.
            self._channels[self._effective_chan_number] = channel_addr
            self._names[self._effective_chan_number] = name
            self._effective_chan_number += 1
    # end connectChannel
    
    def acquire(self) -> int:
        """add one value of each channel into its internal memory,
        if the trigger has been activated.
        Returns 0,1,2 as status
        """
        # Call trigger function
        trigg_value = self._triggFunc()
        # compute trigger status
        if not self._old_trigg_value and trigg_value and not self._trigged:
            self._trigged = True
            # CHANGED from C++ ._acq_counter + self._delay_complement - 1: -1 removed! DOESN'T WORK!
            self._trigged_counter = (self._acq_counter + self._delay_complement - 1) % self._length
            if (self._acq_counter + self._delay_complement - 1) < self._length:
                compute_detail = f'{self._acq_counter}+{self._delay_complement}-1'
            else:
                compute_detail = f'({self._acq_counter}+{self._delay_complement}-1)%{self._length}'
            print(f'  !trigger started: _trigged_counter = {self._trigged_counter} [i.e. {compute_detail}]')

        self._old_trigg_value = trigg_value;
    
        if not self._trigged:
            self._trigged_counter = -1

        print(f'  _acq_counter={self._acq_counter}', end=', ')
        if trigg_value:
            print('trigg_value', end=', ')
        if self._trigged:
            print('_trigged', end=', ')
        print()
        if self._acq_counter != self._trigged_counter:
            self._acq_counter = (self._acq_counter + 1) % self._length
            for k_ch in range(self._effective_chan_number):
                # Python-specific: search self._channels[k_ch] value within self.global_memory:
                self._memory[(self._acq_counter * self._nb_channel) + k_ch] = self.global_memory[self._channels[k_ch]] 
            # CHANGED from C++: _acq_counter updated *after* record
            if self._trigged:
                return 1
            else:
                return 0
        else:
            print(f'  acquisition already done: no record (_final_idx={self._final_idx})')
            self._final_idx = self._acq_counter;
            return 2
    # end acquire
        
    def set_delay(self, d: float):
        """Define a delay to record data before the trigg. delay is in [0, 1]
        """
        if d < 0.0:
            self._delay = 0.0;
        elif d >= 1.0:
            self._delay = self._length
        else:
            self._delay = int(d *self._length)
        
        self._delay_complement = self._length - self._delay
        print(f'Set delay d={d}:')
        print(f'- _delay={self._delay}')
        print(f'- _delay_complement={self._delay_complement}')
    
    def start(self):
        """reset the trigger and enable to record new data_dumped"""
        self._trigged = False
        self._old_trigg_value = False

    def set_trigger(self, func):
        """set trigger function. `func` should return True at some instant to enable trigger
        """
        self._triggFunc = func

### Scope v2

with updated acquisition state machine

![scope acquisition state machine V2](Scope_State-machine.png)

In [3]:
class ScopeStatus(Enum):
    "Scope data acquisition status"
    ACQ_UNTRIG = 0
    ACQ_TRIG = 1
    DONE = 2

In [40]:
class ScopeV2:
    def __init__(self, length: int, nb_channel: int):
        print(f"Init ScopeV2(length={length}, nb_channel={nb_channel})")
        self._length = length
        self._nb_channel = nb_channel
        self._nb_channel_effective = 0
        self._nb_pretrig = 0
        #self._posttrig_delay = 0 # not implemented yet

        # Start state
        self._status = ScopeStatus.ACQ_UNTRIG
        self._acq_count = 0
        self._posttrig_count = 0
        self._mem_idx = 0
        self._last_idx = 0

        self._memory = np.zeros(self._length*self._nb_channel)
        self._names = [f"ch{i}" for i in range(nb_channel)] # default name
        # Pointers to the global memory. 
        # Python-specific adaptation: values should be keys of the `global_memory` dict
        self._channels = [None for i in range(nb_channel)] 

        self._triggFunc = lambda : False
        # Python specific: mockup program global memory where the _channels pointer should point to
        self.global_memory = dict()
    # end init
    
    def connectChannel(self, channel_addr, name: str):
        """store channel address withing (Python-specific: in C it stores the pointer)
        along with channel name
        """
        print(f'Connect channel named "{name}" to variable at addr "{channel_addr}"')
        if self._nb_channel_effective < self._nb_channel:
            # capture the reference of the variable.
            self._channels[self._nb_channel_effective] = channel_addr
            self._names[self._nb_channel_effective] = name
            self._nb_channel_effective += 1
    # end connectChannel
    
    def acquire(self) -> ScopeStatus:
        """add one value of each channel into its internal memory,
        if the trigger has been activated.
        Returns ScopeStatus 0,1,2 as status
        """
        # Call trigger function
        trigg_value = self._triggFunc()
        # Record status state change:
        if self._status == ScopeStatus.ACQ_UNTRIG:
            if trigg_value:
                self._status = ScopeStatus.ACQ_TRIG
                # ACQ_UNTRIG exit: keep at most nb_pretrig samples
                self._acq_count = min(self._acq_count, self._nb_pretrig)
                print(f'  Trigger event! (with already _acq_count={self._acq_count} samples)')
        
        if self._status == ScopeStatus.ACQ_TRIG: # including the case if status was changed just before 
            if self._posttrig_count == self._length - self._nb_pretrig:
                self._status = ScopeStatus.DONE
                # ACQ_TRIG exit: save last record index
                self._last_idx = (self._mem_idx - 1) % length
                print(f'  Record DONE (with _acq_count={self._acq_count}, _final_idx={self._last_idx})')

        # Record action
        if self._status == ScopeStatus.DONE:
            self._status
        else: # ScopeStatus.ACQ_UNTRIG || ScopeStatus.ACQ_TRIG
            print(f'  Recording... (_acq_count={self._acq_count}, _posttrig_count={self._posttrig_count})')
            for k_ch in range(self._nb_channel_effective):
                self._memory[(self._mem_idx * self._nb_channel) + k_ch] = self.global_memory[self._channels[k_ch]] 

            self._mem_idx = (self._mem_idx + 1) % self._length
            self._acq_count = min(self._acq_count + 1, self._length)
            
            if self._status == ScopeStatus.ACQ_TRIG:
                self._posttrig_count = self._posttrig_count + 1

        return self._status
    # end acquire
        
    def set_delay(self, d: float): # TO BE RENAMED set_pretrig(samples: int) and  set_pretrig_ratio(ratio: float)
        """Define a delay to record data before the trigg. delay is in [0, 1]
        """
        if d < 0.0:
            self._nb_pretrig = 0.0;
        elif d >= 1.0:
            self._nb_pretrig = self._length
        else:
            self._nb_pretrig = int(d *self._length)
            
        print(f'Set delay d={d}:')
        print(f'- _nb_pretrig={self._nb_pretrig}')
    
    def start(self):
        """reset the trigger and enable to record new data_dumped"""
        self._status =  ScopeStatus.ACQ_UNTRIG
        self._acq_count = 0
        self._posttrig_count = 0
        self._mem_idx = 0
        self._last_idx = 0

    def set_trigger(self, func):
        """set trigger function. `func` should return True at some instant to enable trigger
        """
        self._triggFunc = func

## Test

### Test trigger functions

1. Always True trigger
2. Trigger which yields values from globally defined `trig_seq` at a globally defined instant `k`
3. Variant of the same idea, but enclosed in a closure: `make_trig_fun(trig_seq)` returns a corresponding trigger function to be used once, since it increments its own time index at each call

In [5]:
def trig_true():
    'always True trigger'
    return True
trig_true()

True

In [6]:
trig_seq = [False, True]
k = 1

In [7]:
def trig_global_seq():
    'trigger which returns the global value trig_seq[k]'
    return trig_seq[k]
trig_global_seq()

True

In [8]:
def make_trig_fun(trig_seq):
    """Returns a triggger function for sequence `trig_seq`
    
    The returned function will sequentially yield the successive values of `trig_seq` at each call
    It will raise a RuntimeError if called more than len(trig_seq) times.
    """
    k = 0
    K = len(trig_seq)
    print(f'creating trig function with sequence of length {K}')
    def trig_fun():
        nonlocal k
        if not k<K:
            raise RuntimeError(f'trig_fun() called more than {K} times')
        t = trig_seq[k]
        k += 1
        return t
    return trig_fun

f = make_trig_fun([False, True, True])
print(f(), f(), f())
f() # should raise RuntimeError

creating trig function with sequence of length 3
False True True


RuntimeError: trig_fun() called more than 3 times

### Init scope

Scope init. Memory is empty (full of 0s) at first

In [9]:
length = 4
nb_channel = 2
scope = ScopeMimicry(length, nb_channel)
scope._memory # size length*nb_channel

Init ScopeMimicry(length=4, nb_channel=2)


array([0., 0., 0., 0., 0., 0., 0., 0.])

Set delay

In [10]:
delay = 0.0
scope.set_delay(delay)

Set delay d=0.0:
- _delay=0
- _delay_complement=4


Connect channels:

In [11]:
print('_effective_chan_number: ', scope._effective_chan_number)
scope.connectChannel('ptr1', 'ch1')
print('_effective_chan_number: ', scope._effective_chan_number)
scope.connectChannel('ptr2', 'ch2')
print('_effective_chan_number: ', scope._effective_chan_number)

_effective_chan_number:  0
Connect channel named "ch1" to variable at addr "ptr1"
_effective_chan_number:  1
Connect channel named "ch2" to variable at addr "ptr2"
_effective_chan_number:  2


Connect trigger

In [12]:
#scope.set_trigger(trig_true) # always True
scope.set_trigger(trig_global_seq) # trigger from globally defined sequence trig_seq
scope._triggFunc

<function __main__.trig_global_seq()>

### Data acquisition test

Test sequences (meant for `length=4`, `delay=0.5`, i.e `_delay=2`):
- trigger at 4th instant
- expected results: store ch1 = 12, 13, 14, 15

In [13]:
ch1_seq = [11, 12, 13, 14, 15]
ch2_seq = [21, 22, 23, 24, 25]
trig_seq = [False, False, False, True, True]
K = len(ch1_seq)

Test sequences (meant for `length=4`, `delay=0.0`):
- trigger at 1set instant
- expected results: store ch1 = 11, 12, 13, 14

In [14]:
ch1_seq = [11, 12, 13, 14, 15]
ch2_seq = [21, 22, 23, 24, 25]
trig_seq = [True, False, False, True, True] # only first True instant should have an impact
K = len(ch1_seq)

Sequence of calls to `scope.acquire()`:

In [15]:
for k in range(K):
    print(f'k={k}: ch1={ch1_seq[k]}, ch2={ch2_seq[k]}')
    scope.global_memory['ptr1'] = ch1_seq[k]
    scope.global_memory['ptr2'] = ch2_seq[k]
    status = scope.acquire()
    print(f'→ return {status},', 'memory=', scope._memory)

k=0: ch1=11, ch2=21
  !trigger started: _trigged_counter = 3 [i.e. 0+4-1]
  _acq_counter=0, trigg_value, _trigged, 
→ return 1, memory= [ 0.  0. 11. 21.  0.  0.  0.  0.]
k=1: ch1=12, ch2=22
  _acq_counter=1, _trigged, 
→ return 1, memory= [ 0.  0. 11. 21. 12. 22.  0.  0.]
k=2: ch1=13, ch2=23
  _acq_counter=2, _trigged, 
→ return 1, memory= [ 0.  0. 11. 21. 12. 22. 13. 23.]
k=3: ch1=14, ch2=24
  _acq_counter=3, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=3)
→ return 2, memory= [ 0.  0. 11. 21. 12. 22. 13. 23.]
k=4: ch1=15, ch2=25
  _acq_counter=3, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=3)
→ return 2, memory= [ 0.  0. 11. 21. 12. 22. 13. 23.]


#### Observations

For experiment with scope of `length=4`, `delay=0.0`, `trig_seq = [1, ....]` (trigger at first instant)
- because `_acq_counter` (index for the circular buffer, incremented with % `length`, i.e. from 0 to `length-1`) is incremented *before* storing, the first data acquired is stored in the 2nd slot.
  - this looks odd, but is not necessarily a bug
- trigger:
    - starts correctly at 1st instant (k=0)
    - is maintained correctly over successive instants
- recording is done as long as `_acq_counter != _trigged_counter`, which is True in two cases:
  - always True if untriggered, because `_trigged_counter = -1`
  - True for some instants once triggered, because `_trigged_counter` gets set to `_acq_counter + _delay_complement - 1` (% `length`, with `_delay_complement= _length - _delay` ∈ [0,... ,`length`]).
    - In that case, `_trigged_counter` = 0+`length`-1 = 3
- BUG: the last element to be recorded is precisely when `_acq_counter` reaches `_trigged_counter`, so the last value of the is missed (only `lenth-1` element recorded, see remaining 0s in the memory)
  - An incorrect fix would be to increase the `_trigged_counter` initialization formula (`_acq_counter + _delay_complement` instead of `_acq_counter + _delay_complement`), however, by virtue of the modulo `length`, this completely blocks the aquisition since then `_acq_counter == _trigged_counter` if `_delay_complement=length` (case of zero delay)

For experiment with scope of `length=4`, `delay=0.5` (i.e `_delay=2`), `trig_seq = [0, 0, 0, 1, 1]`
- trigger starst correctly at 4th instant (k=3)
- same bug as before: missed last data point

### All in one function (for multiple tests)

In [17]:
def test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq, ScopeCls = ScopeMimicry):
    """test scope recording with `length` and `delay` of two channels
    with list data:
    - ch1_seq, ch2_seq: values for ch1, ch2 along instants
    - trig_seq: trigger values along instants
    """
    K = len(ch1_seq)
    assert len(ch1_seq) == len(ch2_seq)
    assert len(ch1_seq) == len(trig_seq)
    print(f'Scope simulating along {K} instants...')
    nb_channel = 2
    scope = ScopeCls(length, nb_channel)
    
    scope.set_delay(delay)
    
    #Connect channels:
    scope.connectChannel('ptr1', 'ch1')
    scope.connectChannel('ptr2', 'ch2')
    
    #Connect trigger
    scope.set_trigger(make_trig_fun(trig_seq))
    
    ### Acquire a sequence of instants: first instant
    
    for k in range(K):
        print(f'k={k}: ch1={ch1_seq[k]}, ch2={ch2_seq[k]}')
        scope.global_memory['ptr1'] = ch1_seq[k]
        scope.global_memory['ptr2'] = ch2_seq[k]
        status = scope.acquire()
        print(f'  → return {status},', 'memory=', scope._memory)
        print()

#### Basic test with 0 delay, trigger at 1st instant

Expected result:
- ch1 recorded values = [11, 12, 13]
- ScopeMimicry: `mem=[13. 23. 11. 21. 12. 22.]`, `final_idx=0`
  - actual result: `mem=[ 0.  0. 11. 21. 12. 22.]`, `final_idx=2` ❌ (last instant missed)
- ScopeV2: `mem=[11. 21. 12. 22. 13. 23.]`, `final_idx=2` → ✅

In [41]:
length = 3
delay = 0.0
ch1_seq = [11, 12, 13, 14, 15]
ch2_seq = [21, 22, 23, 24, 25]
trig_seq = [True, True, True, True, True]

test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq)
print('-'*60)
test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq, ScopeV2)

Scope simulating along 5 instants...
Init ScopeMimicry(length=3, nb_channel=2)
Set delay d=0.0:
- _delay=0
- _delay_complement=3
Connect channel named "ch1" to variable at addr "ptr1"
Connect channel named "ch2" to variable at addr "ptr2"
creating trig function with sequence of length 5
k=0: ch1=11, ch2=21
  !trigger started: _trigged_counter = 2 [i.e. 0+3-1]
  _acq_counter=0, trigg_value, _trigged, 
  → return 1, memory= [ 0.  0. 11. 21.  0.  0.]

k=1: ch1=12, ch2=22
  _acq_counter=1, trigg_value, _trigged, 
  → return 1, memory= [ 0.  0. 11. 21. 12. 22.]

k=2: ch1=13, ch2=23
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [ 0.  0. 11. 21. 12. 22.]

k=3: ch1=14, ch2=24
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [ 0.  0. 11. 21. 12. 22.]

k=4: ch1=15, ch2=25
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_

##### Same, but starting trigger 1 instant later

Expected result:
- ch1 recorded values = [12, 13, 14]
- ScopeMimicry: `mem=[13. 23. 14. 24. 12. 22.]`, `final_idx=1`
  - actual result: `mem=[13. 23. 11. 21. 12. 22.]`, `final_idx=0` ❌ (again last instant missed, keeping instead old instant `k=1`)
- ScopeV2: `mem=[14. 24. 12. 22. 13. 23.]`, `final_idx=0` → ✅

In [42]:
length = 3
delay = 0.0
ch1_seq = [11, 12, 13, 14, 15]
ch2_seq = [21, 22, 23, 24, 25]
trig_seq = [False, True, True, True, True]

test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq)
print('-'*60)
test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq, ScopeV2)

Scope simulating along 5 instants...
Init ScopeMimicry(length=3, nb_channel=2)
Set delay d=0.0:
- _delay=0
- _delay_complement=3
Connect channel named "ch1" to variable at addr "ptr1"
Connect channel named "ch2" to variable at addr "ptr2"
creating trig function with sequence of length 5
k=0: ch1=11, ch2=21
  _acq_counter=0, 
  → return 0, memory= [ 0.  0. 11. 21.  0.  0.]

k=1: ch1=12, ch2=22
  !trigger started: _trigged_counter = 0 [i.e. (1+3-1)%3]
  _acq_counter=1, trigg_value, _trigged, 
  → return 1, memory= [ 0.  0. 11. 21. 12. 22.]

k=2: ch1=13, ch2=23
  _acq_counter=2, trigg_value, _trigged, 
  → return 1, memory= [13. 23. 11. 21. 12. 22.]

k=3: ch1=14, ch2=24
  _acq_counter=0, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [13. 23. 11. 21. 12. 22.]

k=4: ch1=15, ch2=25
  _acq_counter=0, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=0)
  → return 2, memory= [13. 23. 11. 21. 12. 22.]

----------------

#### Example with 100% delay
- bad example, because the expected output is impossible to do, since the trigger starts at the beginning → should record values past the simulation start...

In [43]:
length = 3
delay = 1.0
ch1_seq = [11, 12, 13, 14, 15]
ch2_seq = [21, 22, 23, 24, 25]
trig_seq = [True, True, True, True, True]

test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq)
print('-'*60)
test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq, ScopeV2)

Scope simulating along 5 instants...
Init ScopeMimicry(length=3, nb_channel=2)
Set delay d=1.0:
- _delay=3
- _delay_complement=0
Connect channel named "ch1" to variable at addr "ptr1"
Connect channel named "ch2" to variable at addr "ptr2"
creating trig function with sequence of length 5
k=0: ch1=11, ch2=21
  !trigger started: _trigged_counter = 2 [i.e. 0+0-1]
  _acq_counter=0, trigg_value, _trigged, 
  → return 1, memory= [ 0.  0. 11. 21.  0.  0.]

k=1: ch1=12, ch2=22
  _acq_counter=1, trigg_value, _trigged, 
  → return 1, memory= [ 0.  0. 11. 21. 12. 22.]

k=2: ch1=13, ch2=23
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [ 0.  0. 11. 21. 12. 22.]

k=3: ch1=14, ch2=24
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [ 0.  0. 11. 21. 12. 22.]

k=4: ch1=15, ch2=25
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_

#### Example with 100% delay and late trigger ***TO BE RERUN....***
- expected results: end the record at the trigger instant and yield ch1=11, 12, 13
- ⛓️‍💥 DOESN't work: **several instants after trigger gets recorded**

In [44]:
length = 3
delay = 1.0
ch1_seq = [11, 12, 13, 14, 15, 16]
ch2_seq = [21, 22, 23, 24, 25, 25]
trig_seq = [False, False, False, True, True, True]

test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq)

Scope simulating along 6 instants...
Init ScopeMimicry(length=3, nb_channel=2)
Set delay d=1.0:
- _delay=3
- _delay_complement=0
Connect channel named "ch1" to variable at addr "ptr1"
Connect channel named "ch2" to variable at addr "ptr2"
creating trig function with sequence of length 6
k=0: ch1=11, ch2=21
  _acq_counter=0, 
  → return 0, memory= [ 0.  0. 11. 21.  0.  0.]

k=1: ch1=12, ch2=22
  _acq_counter=1, 
  → return 0, memory= [ 0.  0. 11. 21. 12. 22.]

k=2: ch1=13, ch2=23
  _acq_counter=2, 
  → return 0, memory= [13. 23. 11. 21. 12. 22.]

k=3: ch1=14, ch2=24
  !trigger started: _trigged_counter = 2 [i.e. 0+0-1]
  _acq_counter=0, trigg_value, _trigged, 
  → return 1, memory= [13. 23. 14. 24. 12. 22.]

k=4: ch1=15, ch2=25
  _acq_counter=1, trigg_value, _trigged, 
  → return 1, memory= [13. 23. 14. 24. 15. 25.]

k=5: ch1=16, ch2=25
  _acq_counter=2, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=2)
  → return 2, memory= [13. 23. 14. 24. 15. 25.]



#### Example with short delay (delay=1, length=4) and late trigger
- expected results: record 3 samples after trigger and yield ch1 = 13, 14, 15, 16
- ⛓️‍💥 broken, but like the previous ones (last sample missed)

In [27]:
length = 4
delay = 0.25

ch1_seq  = [11, 12, 13, 14, 15, 16]
ch2_seq  = [21, 22, 23, 24, 25, 25]
trig_seq = [F,  F,  F,  T,  T,  T]

test_2ch_record(length, delay, ch1_seq, ch2_seq, trig_seq)

Scope simulating along 6 instants...
Init ScopeMimicry(length=4, nb_channel=2)
Set delay d=0.25:
- _delay=1
- _delay_complement=3
Connect channel named "ch1" to variable at addr "ptr1"
Connect channel named "ch2" to variable at addr "ptr2"
creating trig function with sequence of length 6
k=0: ch1=11, ch2=21
  _acq_counter=0, 
  → return 0, memory= [ 0.  0. 11. 21.  0.  0.  0.  0.]

k=1: ch1=12, ch2=22
  _acq_counter=1, 
  → return 0, memory= [ 0.  0. 11. 21. 12. 22.  0.  0.]

k=2: ch1=13, ch2=23
  _acq_counter=2, 
  → return 0, memory= [ 0.  0. 11. 21. 12. 22. 13. 23.]

k=3: ch1=14, ch2=24
  !trigger started: _trigged_counter = 1 [i.e. (3+3-1)%4]
  _acq_counter=3, trigg_value, _trigged, 
  → return 1, memory= [14. 24. 11. 21. 12. 22. 13. 23.]

k=4: ch1=15, ch2=25
  _acq_counter=0, trigg_value, _trigged, 
  → return 1, memory= [14. 24. 15. 25. 12. 22. 13. 23.]

k=5: ch1=16, ch2=25
  _acq_counter=1, trigg_value, _trigged, 
  acquisition already done: no record (_final_idx=3)
  → return 2